# ComfyUI Launcher

## Env

In [ ]:

import os
import sys
sys.path.append('.')
os.system('pip install -r ./requirements.txt')
from utils.utils import is_dir_empty, check_gpu
import shutil
import time
import subprocess
from urllib.parse import urlparse
import toml
import git

In [ ]:
# set proxy
from utils.proxy import set_proxy, set_proxy_autodl

# set_proxy('127.0.0.1', 7890)
set_proxy_autodl()

In [ ]:
# config
## path
WORKSPACE_DIR = os.path.join('.','workspace')
WORKSPACE_DIR = os.path.abspath(WORKSPACE_DIR)
MODEL_DIR = os.path.join(WORKSPACE_DIR, 'models')
APP_DIR = os.path.join(WORKSPACE_DIR, 'comfyui')

## app init
APP_INIT_INSTALL_REQ = False

## custom nodes init
CNODES_CONFIG = 'comfyui-custom_nodes.toml'
CNODES_INIT_INSTALL_REQ = True

## model downloading
MODELS_DOWNLOAD = True
MODELS_CONFIG = 'comfyui-models.toml'
MODELS_INCLUDE_CATEGORY = ['sdxl']
MODELS_INCLUDE_TAGS = ['set-lite']

## app launch
AUTO_UPDATE = True
LAUNCH_CLI_ARGS = [
    "--listen 127.0.0.1",
    "--port 8080",
    "--disable-auto-launch"
]
LAUNCH_CLI_ARGS = ' '.join(LAUNCH_CLI_ARGS)

In [ ]:
# debug
## debug env
WORKSPACE_DIR = os.path.join('.', 'workspace', 'test')
# use absolute path
WORKSPACE_DIR = os.path.abspath(WORKSPACE_DIR)
MODEL_DIR = os.path.join(WORKSPACE_DIR, 'models')
APP_DIR = os.path.join(WORKSPACE_DIR, 'comfyui')

CNODES_INIT_INSTALL_REQ = True
MODELS_DOWNLOAD = True
MODELS_INCLUDE_CATEGORY = ['sd15']
AUTO_UPDATE = True

APP_INIT_INSTALL_REQ = True

## Initialize app

In [ ]:
# initialize comfyui

def app_install_req(app_dir=APP_DIR):
    try:
        app_req = ['-r', os.path.join(app_dir, "requirements.txt")]
        if check_gpu() == 'cuda':
            app_req += ['xformers']
        subprocess.run(['pip', 'install'] + app_req, cwd=app_dir, check=True)
        return True
    except Exception as e:
        print(f"🔴 Error: {e}")
        return False

def app_install_comfyui(app_dir=APP_DIR):
    if not os.path.exists(app_dir):
        os.makedirs(app_dir)
    try:
        repo_url = "https://github.com/comfyanonymous/ComfyUI.git"
        git.Repo.clone_from(repo_url, app_dir)
        app_install_req(app_dir)
        return True
    except Exception as e:
        print(f"🔴 Error: {e}")
        return False

def app_init_comfyui(app_dir=APP_DIR, update=AUTO_UPDATE):
    print('📦 Start: initialize comfyui')
    try:
        app_repo = git.Repo(app_dir)
        if app_repo.bare:
            print("📦 ComfyUI broken, reinstalling...")
            shutil.rmtree(app_dir)
            app_install_comfyui(app_dir)
        if app_repo.is_dirty() and update:
            print("🔄 Existed ComfyUI, checking updates...")
            update_result = app_repo.remotes.origin.pull()
            if update_result:
                print("🟢 ComfyUI updated.")
            else:
                print("🟡 ComfyUI already up to date.")
        if APP_INIT_INSTALL_REQ:
            app_install_req(app_dir)
        return True
    except git.exc.InvalidGitRepositoryError:
        print("📦 ComfyUI is broken, reinstalling...")
        shutil.rmtree(app_dir)
        app_install_comfyui(app_dir)
        return True
    except git.exe.NoSuchPathError:
        print("📦 ComfyUI not found, installing...")
        app_install_comfyui(app_dir)
        return True
    except Exception as e:
        print(f"🔴 Error: {e}")
        return False

app_init_comfyui()

## Download models

In [ ]:
# prepare models dir
# !mkdir -p $MODEL_DIR/configs
# !cp -rf $APP_DIR/models/configs/* $MODEL_DIR/configs
# !rm -rf $APP_DIR/models
# !ln -sf $MODEL_DIR $APP_DIR/models
os.makedirs(os.path.join(MODEL_DIR), exist_ok=True)
if not os.path.islink(os.path.join(APP_DIR, "models")):
    src = MODEL_DIR
    dst = os.path.join(APP_DIR, "models")
    if os.path.exists(os.path.join(dst, "configs")) and not is_dir_empty(dst) and not os.path.exists(os.path.join(src, "configs")):
        shutil.copytree(os.path.join(dst, "configs"), os.path.join(src, "configs"), dirs_exist_ok=True)
    shutil.rmtree(os.path.join(dst))
    os.symlink(src, dst, target_is_directory=True)

In [ ]:
from utils.download import download_models

# exec models download
if MODELS_DOWNLOAD:
    config_models = toml.load(MODELS_CONFIG)
    models = config_models['models']
    download_models(models, MODEL_DIR, MODELS_INCLUDE_CATEGORY, MODELS_INCLUDE_TAGS)

## Initialize `custom_nodes`

In [ ]:
def init_custom_nodes(custom_nodes, install_requirements=CNODES_INIT_INSTALL_REQ, max_retries=3, retry_interval=5):
    print(f"📦 Start: initializing custom nodes")
    requirements_list = []
    new_list = []
    existing_list = []
    updated_list = []
    error_list = []
    for cn in custom_nodes:
        cn_url = cn['url']
        cn_name = urlparse(cn_url).path.split('/')[-1].split('.')[0]
        cn_path = os.path.join(APP_DIR, "custom_nodes", cn_name)
        if os.path.exists(os.path.join(cn_path, '.git')) and len(os.listdir(cn_path)) == 1:
            print(f"🔴 {cn_name}: custom nodes broken, will reinstall")
            shutil.rmtree(cn_path)
        retries = 0
        while retries < max_retries:
            try:
                if os.path.exists(cn_path) and not is_dir_empty(cn_path):
                    print(f"🟡 {cn_name}: existed custom nodes.")
                    existing_list.append(cn_name)
                    if AUTO_UPDATE:
                        update_result = subprocess.run(['git', 'pull'], cwd=cn_path, capture_output=True, text=True, check=True)
                        if 'Already up to date' in update_result.stdout:
                            print(f"🟡 {cn_name}: custom nodes already up to date.")
                            break
                        else:
                            print(f"🟢 {cn_name}: custom nodes updated.")
                            updated_list.append(cn_name)
                            break
                else:
                    print(f"🟢 {cn_name}: new custom nodes, cloning...")
                    subprocess.check_call(['git', 'clone', cn_url, cn_path])
                    requirements_list + ["-r", os.path.join(cn_path, "requirement.txt")]
                    new_list.append(cn_name)
                    break
            except Exception as e:
                print(f"🔴 {cn_name}: error: {e}")
                retries += 1
                print(f"⏳ Retrying, sleep {retry_interval}s... ({retries}/{max_retries})")
                time.sleep(retry_interval)
        if retries >= max_retries:
            print(f"🔴 {cn_name}: failed to clone!")
            error_list.append(cn_name)
            return False
        
    total_counts = len(custom_nodes)
    existing_counts =len(existing_list)
    new_counts = len(new_list)
    error_counts = len(error_list)
    print(f"📦 Finished: initialize custom nodes")
    print(f"📦 Total: {total_counts}")
    print(f"🟡 Existed: {existing_counts}")
    if AUTO_UPDATE:
        updated = len(updated_list)
        print(f"🟢 Updated: {updated}")
    print(f"🟢 New: {new_counts}")
    if new_counts > 0:
        for i in new_list:
            print(f"- {i}")
    print(f"🔴 Error: {error_counts}")
    if error_counts > 0:
        for i in error_list:
            print(f" - {i}")

    if install_requirements and requirements_list:
        print(f"📦 Try: pre-install new custom nodes requirements")
        install_requirements_command = ["pip", "install"] + requirements_list
        try:
            subprocess.run(install_requirements_command, cwd=APP_DIR, check=True)
        except Exception as e:
            print(f"🔴 Error: {e}")
            return False
        print(f"🟢 Success: pre-install new custom nodes requirements.")

    return True

In [ ]:
# initial comfyui custom_nodes
config_custom_nodes = toml.load(CNODES_CONFIG)
custom_nodes = config_custom_nodes['custom_nodes']
custom_nodes_models = config_custom_nodes['models']
# download custom_nodes from list
init_custom_nodes(custom_nodes)
download_models(custom_nodes_models, MODEL_DIR, MODELS_INCLUDE_CATEGORY, MODELS_INCLUDE_TAGS)

## Launch

In [ ]:
!python $APP_DIR/main.py {LAUNCH_CLI_ARGS}

## Utils

In [ ]:
# shutdown
os.system("/usr/bin/shutdown")

In [ ]:
# reinstall requirements
subprocess.run(['pip', 'install', '-r', "requirements.txt"], cwd=APP_DIR, check=True)